In [2]:
import json
import os
import pandas as pd
from tqdm import tqdm
import re
pd.options.display.max_colwidth = 70

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/ML4CC General-GOOD.json"

import fsspec
fs = fsspec.filesystem("gs")

metas = []
for meta_path in tqdm(fs.glob("gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/*/meta/*.json")):
    with fs.open(f"gs://{meta_path}") as fh:
        datadict = json.load(fh)
    
    datadict["layer name"] = os.path.splitext(os.path.basename(meta_path))[0]
    datadict["event id"] = datadict["layer name"].replace("observed_event_a","")
    datadict["splited_file"] = re.search("\d{10}-\d{10}", datadict["layer name"])
    datadict["meta_path"] = meta_path
    datadict["split"] = meta_path.split("/")[-3]
    metas.append(datadict)
        
metas_pd = pd.DataFrame(metas)
metas_pd

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 461/461 [03:00<00:00,  2.55it/s]


source     satellite        satellite date         shape  \
0    CopernicusEMS    Sentinel-2  2018-05-17T15:36:15Z  [2643, 2170]   
1    CopernicusEMS    Sentinel-2  2018-05-17T15:36:15Z   [1012, 420]   
2    CopernicusEMS    Sentinel-1  2018-11-03T17:03:57Z   [793, 1376]   
3    CopernicusEMS    Sentinel-1  2018-11-03T17:03:57Z  [1391, 1766]   
4    CopernicusEMS    Sentinel-1  2018-11-04T05:04:59Z    [208, 124]   
..             ...           ...                   ...           ...   
456  CopernicusEMS    Sentinel-1  2018-04-13T17:52:00Z  [3881, 7286]   
457  CopernicusEMS  COSMO-SkyMed  2018-05-13T16:39:37Z  [3918, 9554]   
458  CopernicusEMS      SPOT-6-7  2018-05-26T10:13:43Z   [798, 1602]   
459         unosat    RADARSAT-2  2016-10-08T00:00:00Z    [997, 977]   
460         unosat    Sentinel-1  2016-10-14T00:00:00Z   [3390, 910]   

     pixels invalid S2  pixels clouds S2  pixels water S2  pixels land S2  \
0               145955           3031674           188064         2369617   
1                    0             59267            27858          337915   
2               213171             25221            16551          836225   
3                 1560            488750           137302         1828894   
4                    0                 0              502           25290   
..                 ...               ...              ...             ...   
456             731298           9281529           976777        17287362   
457             351185              9947          4900316        32171124   
458                  0               841            32860         1244695   
459                485            218547            46108          708929   
460             221015            510478            75712         2277695   

     pixels flood water S2  pixels hydro water S2  ...  \
0                   138397                  43763  ...   
1                    14385                   9231  ...   
2                     9829                   6660  ...   
3                    63684                  11291  ...   
4                      502                      0  ...   
..                     ...                    ...  ...   
456                 785645                 133867  ...   
457                 301084                 672106  ...   
458                   1483                  10063  ...   
459                  46108                      0  ...   
460                  34193                      0  ...   

                                                        layer name  \
0           EMSR286_08ITUANGONORTH_DEL_MONIT02_v1_observed_event_a   
1           EMSR286_09ITUANGOSOUTH_DEL_MONIT02_v1_observed_event_a   
2              EMSR333_01RATTALORO_DEL_MONIT01_v1_observed_event_a   
3              EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a   
4    EMSR333_13TORRECOLONNASPERONE_DEL_MONIT01_v2_observed_event_a   
..                                                             ...   
456             EMSR279_05ZARAGOZA_DEL_MONIT02_v1_observed_event_a   
457                EMSR280_03FALUN_DEL_MONIT06_v2_observed_event_a   
458                      EMSR287_05MAGWITZ_DEL_v1_observed_event_a   
459                        RS2_20161008_Water_Extent_Corail_Pestel   
460                         ST1_20161014_WaterExtent_BinhDinh_Lake   

                                          event id splited_file  \
0           EMSR286_08ITUANGONORTH_DEL_MONIT02_v1_         None   
1           EMSR286_09ITUANGOSOUTH_DEL_MONIT02_v1_         None   
2              EMSR333_01RATTALORO_DEL_MONIT01_v1_         None   
3              EMSR333_02PORTOPALO_DEL_MONIT01_v1_         None   
4    EMSR333_13TORRECOLONNASPERONE_DEL_MONIT01_v2_         None   
..                                             ...          ...   
456             EMSR279_05ZARAGOZA_DEL_MONIT02_v1_         None   
457                EMSR280_03FALUN_DEL_MONIT06_v2_         None   
458                      EMSR287_05MAGWITZ_DEL_v1_         None   
459        RS2

In [ ]:
metas_pd.bounds

In [9]:
metas_pd["splited_file"] = metas_pd["layer name"].apply(lambda x: re.search("\d{10}-\d{10}", x) is not None)
metas_pd["floodmap_path"] = metas_pd["meta_path"].apply(lambda x: x.replace("/meta/","/floodmaps/").replace("json","geojson"))
cols_interesting = ["shape", "satellite", "layer name", "percentage clouds S2", "percentage water S2", "percentage land S2", "split", "satellite date","bounds","splited_file", "floodmap_path", "source"]

metas_pd["total_pixels_valid"] = metas_pd["pixels clouds S2"] + metas_pd["pixels water S2"] + metas_pd["pixels land S2"]
metas_pd["percentage clouds S2"]= metas_pd["pixels clouds S2"] / metas_pd["total_pixels_valid"] * 100
metas_pd["percentage water S2"]= metas_pd["pixels water S2"] / metas_pd["total_pixels_valid"] * 100
metas_pd["percentage land S2"]= metas_pd["pixels land S2"] / metas_pd["total_pixels_valid"] * 100
metas_glofimr = metas_pd.loc[metas_pd.source == "glofimr", cols_interesting].copy()
metas_glofimr["ems_code"] = metas_glofimr["layer name"].apply(lambda x: f"GLOFIMR{x.split('_')[0]}")
metas_glofimr["aoi_code"] = metas_glofimr["layer name"].apply(lambda x: f"_".join(x.split('_')[1:]))
metas_glofimr

shape          satellite  \
11   [10086, 8742]          Landsat-8   
12    [1980, 2841]          Landsat-8   
13    [4095, 3447]          Landsat-8   
14   [11579, 9067]         Sentinel-1   
15    [4379, 6553]         Sentinel-1   
16  [12544, 12082]          Landsat-8   
17    [326, 12082]          Landsat-8   
18  [12496, 10659]          Landsat-8   
19     [989, 1634]          Landsat-8   
20    [1383, 4514]          Landsat-8   
21    [1036, 2493]          Landsat-8   
22    [9783, 7363]         Sentinel-1   
23    [3312, 2738]         Sentinel-1   
24    [9647, 6201]  earth observing 1   
25   [6674, 12544]          Landsat-8   
26    [6674, 1472]          Landsat-8   
27     [597, 9370]          Landsat-8   
28    [8924, 5759]          Landsat-8   
29    [3531, 8125]          Landsat-8   
30    [7164, 6869]          Landsat-8   

                                                             layer name  \
11                       01042016_Choctawhatchee_River_near_Bellwood_AL   
12                                   01042016_Holmes_Creek_at_Vernon_FL   
13                                    01042016_Pea_River_near_Samson_AL   
14                                 05042017_Black_River_near_Corning_AR   
15                                     05042017_Cache_River_at_Egypt_AR   
16         05102017_Black_River_near_Pocahontas_AR0000000000-0000000000   
17         05102017_Black_River_near_Pocahontas_AR0000012544-0000000000   
18                              05102017_Mississippi_River_at_Thebes_IL   
19                              05282016_Brazos_River_near_Hempstead_TX   
20                                05282016_Spring_Creek_near_Tomball_TX   
21                                05282016_Willow_Creek_near_Tomball_TX   
22                                 05302016_Brazos_River_at_Richmond_TX   
23                              05302016_San_Jacinto_River_at_Porter_TX   
24                         08162016_Mississippi_River_at_Baton_Rouge_LA   
25       09262016_Maquoketa_River_at_Manchester_IA0000000000-0000000000   
26       09262016_Maquoketa_River_at_Manchester_IA0000000000-0000012544   
27  09262016_Wapsipinicon_River_at_Independence_IA0000012544-0000012544   
28                       10132016_Ashley_River_near_North_Charleston_SC   
29                                 10132016_Black_River_at_Kingstree_SC   
30                             10132016_Cooper_River_at_Monks_Corner_SC   

    percentage clouds S2  percentage water S2  percentage land S2  split  \
11              0.027975             8.358769           91.613257  train   
12              0.003359             6.743588           93.253053  train   
13              0.021773             5.269413           94.708814  train   
14              0.017562             4.174417           95.808022  train   
15              0.034505             4.939790           95.025705  train   
16              0.022405             4.408495           95.569100  train   
17              0.158324             0.891043           98.950633  train   
18              5.495384            11.198859           83.305757  train   
19             56.108379            17.014454           26.877167  train   
20             99.533002             0.051559            0.415439  train   
21            100.000000             0.000000            0.000000  train   
22             92.626790             0.367848            7.005362  train   
23             97.386971             0.225132            2.387897  train   
24             30.713520             2.897530           66.388950  train   
25             30.878251             0.507030           68.614719  train   
26            100.000000             0.000000            0.000000  train   
27             83.121007             0.100094           16.778899  train   
28              0.844571            17.977371           81.178058  train   
29              0.093993            10.965083           88.940924  train   
30              1.538692            13.949334          

In [10]:
metas_glofimr_good = metas_glofimr.loc[(metas_glofimr["percentage clouds S2"] < 80) & ~metas_glofimr["splited_file"], :]
metas_glofimr_good

shape          satellite  \
11   [10086, 8742]          Landsat-8   
12    [1980, 2841]          Landsat-8   
13    [4095, 3447]          Landsat-8   
14   [11579, 9067]         Sentinel-1   
15    [4379, 6553]         Sentinel-1   
18  [12496, 10659]          Landsat-8   
19     [989, 1634]          Landsat-8   
24    [9647, 6201]  earth observing 1   
28    [8924, 5759]          Landsat-8   
29    [3531, 8125]          Landsat-8   
30    [7164, 6869]          Landsat-8   

                                        layer name  percentage clouds S2  \
11  01042016_Choctawhatchee_River_near_Bellwood_AL              0.027975   
12              01042016_Holmes_Creek_at_Vernon_FL              0.003359   
13               01042016_Pea_River_near_Samson_AL              0.021773   
14            05042017_Black_River_near_Corning_AR              0.017562   
15                05042017_Cache_River_at_Egypt_AR              0.034505   
18         05102017_Mississippi_River_at_Thebes_IL              5.495384   
19         05282016_Brazos_River_near_Hempstead_TX             56.108379   
24    08162016_Mississippi_River_at_Baton_Rouge_LA             30.713520   
28  10132016_Ashley_River_near_North_Charleston_SC              0.844571   
29            10132016_Black_River_at_Kingstree_SC              0.093993   
30        10132016_Cooper_River_at_Monks_Corner_SC              1.538692   

    percentage water S2  percentage land S2  split        satellite date  \
11             8.358769           91.613257  train  2016-01-04T00:00:00Z   
12             6.743588           93.253053  train  2016-01-04T00:00:00Z   
13             5.269413           94.708814  train  2016-01-04T00:00:00Z   
14             4.174417           95.808022  train  2017-05-04T00:00:00Z   
15             4.939790           95.025705  train  2017-05-04T00:00:00Z   
18            11.198859           83.305757  train  2017-05-10T00:00:00Z   
19            17.014454           26.877167  train  2016-05-28T00:00:00Z   
24             2.897530           66.388950  train  2016-08-16T00:00:00Z   
28            17.977371           81.178058  train  2016-10-13T00:00:00Z   
29            10.965083           88.940924  train  2016-10-13T00:00:00Z   
30            13.949334           84.511974  train  2016-10-13T00:00:00Z   

                                                                   bounds  \
11  [-86.43742377396336, 30.3493123793928, -85.65211655258608, 31.2553...   
12  [-85.87804284654213, 30.53256869735318, -85.62283147432377, 30.710...   
13  [-86.16783935719909, 30.962322729275957, -85.85819007876309, 31.33...   
14  [-90.95846493588009, 36.25052514383076, -90.14396246776892, 37.290...   
15  [-90.7714356937264, 36.214053543295506, -90.18276968804288, 36.607...   
18  [-89.92046162507998, 35.73058025738238, -88.96294736373699, 36.853...   
19  [-96.22789256099679, 30.136231993999647, -96.08110784357166, 30.22...   
24  [-91.41418027951391, 30.046939454758167, -90.8571349718314, 30.913...   
28  [-80.38709067236157, 32.55889848374158, -79.86975090023714, 33.360...   
29  [-80.062709023266, 33.42298795553615, -79.33282785491889, 33.74018...   
30  [-80.23904831353866, 32.79829950695944, -79.62199554487697, 33.441...   

    splited_file  \
11         False   
12         False   
13         False   
14         False   
15         False   
18         False   
19         False   
24         False   
28         False   
29         False   
30         False   

                                                            floodmap_path  \
11  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/0104...   
12  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/0104...   
13  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/0104...   
14  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/0504...   
15  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/0504...   
18  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/

In [12]:
from shapely.geometry import Polygon
from ml4floods.data.ee_download import generate_polygon
from datetime import datetime
from ml4floods.data import utils


for idx,t in metas_glofimr_good.iterrows():
    bounding_box = dict(zip(["west", "south","east", "north"],t["bounds"]))
    area_of_interest_polygon =  Polygon(generate_polygon(t["bounds"])[0])
    ems_code = t["ems_code"]
    aoi_code = t["aoi_code"]
    satellite_date = datetime.fromisoformat(t["satellite date"].replace("Z","+00:00"))
    data_save = {'event id': t["layer name"],
                 'layer name': t["layer name"],
                 'event type': 'Flood',
                 'satellite date': satellite_date,
                 'satellite': t["satellite"],
                 'bounding box':bounding_box,
                 'reference system': 'epsg:4326',
                 'source': t["source"],
                 'area_of_interest_polygon': area_of_interest_polygon,
                 'ems_code': ems_code,
                 'aoi_code': aoi_code,
                 'date_ems_code': satellite_date}
    floodmap_path = f"gs://{t['floodmap_path']}"
    print(floodmap_path)
    assert fs.exists(floodmap_path), f"{floodmap_path} does not exist"
    date_file = satellite_date.strftime("%Y-%m-%d")
    ds_path = f"gs://ml4cc_data_lake/0_DEV/1_Staging/WorldFloods/{ems_code}/{aoi_code}"
    pickle_path = f"{ds_path}/flood_meta/{date_file}.pickle"
    floodmap_path_dest = f"{ds_path}/floodmap/{date_file}.geojson"
    utils.write_pickle_to_gcp(pickle_path, data_save)
    fs.copy(floodmap_path, floodmap_path_dest)

gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/01042016_Choctawhatchee_River_near_Bellwood_AL.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/01042016_Holmes_Creek_at_Vernon_FL.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/01042016_Pea_River_near_Samson_AL.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/05042017_Black_River_near_Corning_AR.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/05042017_Cache_River_at_Egypt_AR.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/05102017_Mississippi_River_at_Thebes_IL.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/05282016_Brazos_River_near_Hempstead_TX.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/08162016_Mississippi_River_at_Baton_Rouge_LA.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/10132016_Ashley_River_

In [6]:
data = utils.read_pickle_from_gcp("gs://ml4cc_data_lake/0_DEV/1_Staging/WorldFloods/EMSR333/02PORTOPALO/flood_meta/2018-11-04.pickle")
data

{'event id': 'EMSR333_02PORTOPALO_DEL_MONIT01_v1',
 'layer name': 'EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a',
 'event type': 'Flash flood',
 'satellite date': Timestamp('2018-11-04 09:51:59+0000', tz='UTC'),
 'country': 'NaN',
 'satellite': 'Sentinel-2',
 'bounding box': {'west': 12.857748170000036,
  'east': 13.016296379000039,
  'north': 37.69603375300005,
  'south': 37.57122292200006},
 'reference system': 'epsg:4326',
 'abstract': 'NaN',
 'purpose': 'NaN',
 'source': 'CopernicusEMS',
 'area_of_interest_polygon': <shapely.geometry.polygon.Polygon at 0x7fafbd0f3be0>,
 'observed_event_file': 'EMSR333_02PORTOPALO_DEL_MONIT01_v1_observed_event_a.shp',
 'area_of_interest_file': 'EMSR333_02PORTOPALO_DEL_MONIT01_v1_area_of_interest_a.shp',
 'ems_code': 'EMSR333',
 'aoi_code': '02PORTOPALO',
 'date_ems_code': datetime.datetime(2018, 11, 2, 0, 0, tzinfo=datetime.timezone.utc),
 'satellite_pre_event': 'Sentinel-2',
 'timestamp_pre_event': Timestamp('2018-10-30 09:57:21+0000', tz='U

In [13]:
metas_unosat = metas_pd.loc[metas_pd.source == "unosat", cols_interesting].copy()

metas_unosat["ems_code"] = metas_unosat["layer name"].apply(lambda x: f"UNOSAT{x.replace('Water_Extent','WaterExtent').split('_WaterExtent_')[0]}")
metas_unosat["aoi_code"] = metas_unosat["layer name"].apply(lambda x: f"_".join(x.replace('Water_Extent','WaterExtent').split('_WaterExtent_')[1:]))
metas_unosat

shape   satellite  \
372  [12544, 12544]   Landsat-8   
373  [12544, 11838]   Landsat-8   
374  [12544, 12544]   Landsat-8   
375  [12544, 11838]   Landsat-8   
376   [1643, 11838]   Landsat-8   
..              ...         ...   
452  [12544, 12544]  TERRASAR-X   
453   [4299, 12544]  TERRASAR-X   
454    [6397, 4308]  TERRASAR-X   
459      [997, 977]  RADARSAT-2   
460     [3390, 910]  Sentinel-1   

                                                              layer name  \
372                LS20161216_Water_Extent_LakeChad0000000000-0000000000   
373                LS20161216_Water_Extent_LakeChad0000000000-0000012544   
374                LS20161216_Water_Extent_LakeChad0000012544-0000000000   
375                LS20161216_Water_Extent_LakeChad0000012544-0000012544   
376                LS20161216_Water_Extent_LakeChad0000025088-0000012544   
..                                                                   ...   
452  TX_20170816_WaterExtent_NorthWesternBangladesh0000000000-0000000000   
453  TX_20170816_WaterExtent_NorthWesternBangladesh0000012544-0000000000   
454                         TX_TDX_20180727_WaterExtent_SanamxayDistrict   
459                              RS2_20161008_Water_Extent_Corail_Pestel   
460                               ST1_20161014_WaterExtent_BinhDinh_Lake   

     percentage clouds S2  percentage water S2  percentage land S2  split  \
372              0.482561            14.853872           84.663567  train   
373              0.602410             4.789171           94.608419  train   
374              1.154662             4.213061           94.632277  train   
375              0.655542            12.459497           86.884961  train   
376              0.000422             0.490773           99.508805  train   
..                    ...                  ...                 ...    ...   
452            100.000000             0.000000            0.000000  train   
453             99.899449             0.000000            0.100551  train   
454             98.975119             0.098839            0.926042  train   
459             22.447678             4.735904           72.816419    val   
460             17.824668             2.643682           79.531650    val   

           satellite date  \
372  2016-12-16T00:00:00Z   
373  2016-12-16T00:00:00Z   
374  2016-12-16T00:00:00Z   
375  2016-12-16T00:00:00Z   
376  2016-12-16T00:00:00Z   
..                    ...   
452  2017-08-16T00:00:00Z   
453  2017-08-16T00:00:00Z   
454  2018-07-27T00:00:00Z   
459  2016-10-08T00:00:00Z   
460  2016-10-14T00:00:00Z   

                                                                    bounds  \
372  [13.162385037504464, 13.265152306007737, 14.289231729903992, 14.39...   
373  [14.289231729903992, 13.265152306007737, 15.352657363244681, 14.39...   
374  [13.162385037504464, 12.138305613608209, 14.289231729903992, 13.26...   
375  [14.289231729903992, 12.138305613608209, 15.352657363244681, 13.26...   
376  [14.289231729903992, 11.990712412427373, 15.352657363244681, 12.13...   
..                                                                     ...   
452  [89.06113322429124, 25.425915801718936, 90.18797991669076, 26.5527...   
453  [89.06113322429124, 25.039730061075954, 90.18797991669076, 25.4259...   
454  [106.30276796698246, 14.505276555734737, 106.68976219138115, 15.07...   
459  [-73.9251495075762, 18.449239979133083, -73.83738410431772, 18.538...   
460  [108.75489919804352, 14.023779563446674, 108.83664588889839, 14.32...   

     splited_file  \
372          True   
373          True   
374          True   
375          True   
376          True   
..            ...   
452          True   
453          True   
454         False   
459         False   
460         False   

                                                             floodmap_path  \
372  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/LS20...   
373  ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/flo

In [14]:
metas_unosat_good = metas_unosat.loc[(metas_unosat["percentage clouds S2"] < 80) & ~metas_unosat["splited_file"], :].copy()
metas_unosat_good['ems_code'].iloc[0] = "UNOSATLS20170416"
metas_unosat_good['aoi_code'].iloc[0] = "SouthUrmiaLake"
metas_unosat_good["ems_code"] = metas_unosat_good["ems_code"].apply(lambda x: x.replace("_",""))
metas_unosat_good

/home/gonzalo/miniconda3/envs/ml4floods/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


shape    satellite  \
377    [1162, 2210]    Landsat-8   
385    [9228, 8526]   RADARSAT-2   
388  [11999, 11751]   RADARSAT-2   
422   [8782, 12442]   Sentinel-1   
423      [984, 469]   Sentinel-1   
429    [1594, 2268]  WorldView-2   
434    [5124, 5991]   Sentinel-1   
440    [5951, 9462]   Sentinel-1   
441    [5791, 7592]   Sentinel-1   
448    [4625, 5957]   TERRASAR-X   
451    [5851, 3645]   TERRASAR-X   
459      [997, 977]   RADARSAT-2   
460     [3390, 910]   Sentinel-1   

                                                 layer name  \
377  LS20170416_SouthUrmiaLake_SatelliteDetectedWaterExtent   
385                      RS2_20170122_WaterExtent_SaveRiver   
388                     RS2_20170925_WaterExtent_Lai_Bangor   
422                   ST20180507_WaterExtent_Moibalo_Somali   
423                           ST20180522_WaterExtent_Teknaf   
429              ST_20170426_WaterExtent_LesCayes_Cavaillon   
434                       ST_20170811_WaterExtent_Khamkeuth   
440       ST_20170823_WaterExtent_AmuruAndAdjumaniDistricts   
441                          ST_20170826_WaterExtent_Niamey   
448                   TSX_20170308_WaterExtent_Maroantsetra   
451                TX_20160811_WaterExtent_WetSoil_Khartoum   
459                 RS2_20161008_Water_Extent_Corail_Pestel   
460                  ST1_20161014_WaterExtent_BinhDinh_Lake   

     percentage clouds S2  percentage water S2  percentage land S2  split  \
377              1.068373            15.298085           83.633542  train   
385             53.868835             5.435786           40.695379  train   
388             32.495530             1.046166           66.458303  train   
422             27.156840             0.726168           72.116992  train   
423             60.548909             9.225550           30.225541  train   
429             58.815235            20.462710           20.722055  train   
434             45.132075             2.212668           52.655257  train   
440             18.424270             0.897587           80.678143  train   
441             54.584462             1.123558           44.291980  train   
448             72.660413             8.608375           18.731212  train   
451             55.672447             5.180081           39.147471  train   
459             22.447678             4.735904           72.816419    val   
460             17.824668             2.643682           79.531650    val   

           satellite date  \
377  2017-04-16T00:00:00Z   
385  2017-01-22T00:00:00Z   
388  2017-09-25T00:00:00Z   
422  2018-05-07T00:00:00Z   
423  2018-05-22T00:00:00Z   
429  2017-04-26T00:00:00Z   
434  2017-08-11T00:00:00Z   
440  2017-08-23T00:00:00Z   
441  2017-08-26T00:00:00Z   
448  2017-03-08T00:00:00Z   
451  2016-08-11T00:00:00Z   
459  2016-10-08T00:00:00Z   
460  2016-10-14T00:00:00Z   

                                                                    bounds  \
377  [45.513233701443966, 36.94797713042115, 45.71176137923438, 37.0523...   
385  [34.35822399783299, -21.53351567562905, 35.12412760907329, -20.704...   
388  [15.892994006642574, 9.106042372062767, 16.948604297011425, 10.183...   
422  [43.922047838683056, 4.891057227445558, 45.039731715184566, 5.6799...   
423  [92.23056920972174, 20.917181559194646, 92.27270019654694, 21.0055...   
429  [-73.80747020535654, 18.18073354070976, -73.60373229891823, 18.323...   
434  [104.6563357142482, 17.931990038537066, 105.1945164009642, 18.3922...   
440  [31.529159673555377, 3.294481472979933, 32.37914559538927, 3.82906...   
441  [1.571153431925043, 13.358127937914109, 2.2531543956285836, 13.878...   
448  [49.490794116468386, -15.624577399747661, 50.02592053121838, -15.2...   
451  [32.42325287583954, 15.31627559423784, 32.7506887969011, 15.841879...   
459  [-73.9251495075762, 18.449239979133083, -73.83738410431772, 18.538...   
460  [108.75489919804352, 14.023779563446674, 108.83664588889839, 14.32...   

     splited_file  \
377         False   
385  

In [15]:
from shapely.geometry import Polygon
from ml4floods.data.ee_download import generate_polygon
from datetime import datetime
from ml4floods.data import utils


for idx,t in metas_unosat_good.iterrows():
    bounding_box = dict(zip(["west", "south","east", "north"],t["bounds"]))
    area_of_interest_polygon =  Polygon(generate_polygon(t["bounds"])[0])
    ems_code = t["ems_code"]
    aoi_code = t["aoi_code"]
    satellite_date = datetime.fromisoformat(t["satellite date"].replace("Z","+00:00"))
    data_save = {'event id': t["layer name"],
                 'layer name': t["layer name"],
                 'event type': 'Flood',
                 'satellite date': satellite_date,
                 'satellite': t["satellite"],
                 'bounding box':bounding_box,
                 'reference system': 'epsg:4326',
                 'source': t["source"],
                 'area_of_interest_polygon': area_of_interest_polygon,
                 'ems_code': ems_code,
                 'aoi_code': aoi_code,
                 'date_ems_code': satellite_date}
    floodmap_path = f"gs://{t['floodmap_path']}"
    print(floodmap_path)
    assert fs.exists(floodmap_path), f"{floodmap_path} does not exist"
    date_file = satellite_date.strftime("%Y-%m-%d")
    ds_path = f"gs://ml4cc_data_lake/0_DEV/1_Staging/WorldFloods/{ems_code}/{aoi_code}"
    pickle_path = f"{ds_path}/flood_meta/{date_file}.pickle"
    floodmap_path_dest = f"{ds_path}/floodmap/{date_file}.geojson"
    utils.write_pickle_to_gcp(pickle_path, data_save)
    fs.copy(floodmap_path, floodmap_path_dest)

gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/LS20170416_SouthUrmiaLake_SatelliteDetectedWaterExtent.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/RS2_20170122_WaterExtent_SaveRiver.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/RS2_20170925_WaterExtent_Lai_Bangor.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/ST20180507_WaterExtent_Moibalo_Somali.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/ST20180522_WaterExtent_Teknaf.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/ST_20170426_WaterExtent_LesCayes_Cavaillon.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/ST_20170811_WaterExtent_Khamkeuth.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/ST_20170823_WaterExtent_AmuruAndAdjumaniDistricts.geojson
gs://ml4cc_data_lake/0_DEV/2_Mart/worldfloods_v1_0/train/floodmaps/ST_20170826_